In [59]:
import pandas as pd
import re
import logging
import os

PATH_TO_DATA = '/Users/anne/surfdrive/uva/projects/RPA_KeepingScore/data/'
FILE = 'parliamentary_questions_parsed.pkl'

In [56]:
def parse_parliamentary_questions():
    df = pd.read_pickle(PATH_TO_DATA + FILE)
    df['text'] = df['questions'].astype(str) + ' ' + df['answers'].astype(str)
    text = df['text'].to_list()
    sentence = [ re.split("(?<=[.!?])\s+", element) for element in text ]
    sentences = [item for sublist in sentence for item in sublist]
    logging.info("The total length of sentences is: {}...".format(len(sentences)))
    sentences = set(sentences)
    logging.info("...After removing duplicates, the length of the sentences is: {}".format(len(sentences)) )
    return sentences

In [57]:
def get_sentences():
    
    sentences = parse_parliamentary_questions()
    
    filename = '{}sentences_parliamentary_questions.txt'.format(PATH_TO_DATA)
    dirname = os.path.dirname(filename)
    if not os.path.exists(dirname):
        os.makedirs(dirname)
   
    with open(filename, mode='w') as fo:
        for s in sentences:
            fo.write(s)
            fo.write('\n')
            
    print("Created file with sentences: {}".format(filename))

In [60]:
if __name__ == "__main__":

    logger = logging.getLogger()
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
    logging.root.setLevel(level=logging.INFO)

    get_sentences()

2019-09-23 10:34:37,916 : INFO : The total length of sentences is: 1814472...
2019-09-23 10:34:39,164 : INFO : ...After removing duplicates, the length of the sentences is: 1525810


Created file with sentences: /Users/anne/surfdrive/uva/projects/RPA_KeepingScore/data/sentences_parliamentary_questions.txt


In [61]:
df = pd.read_pickle(PATH_TO_DATA + 'VK_TEL_merged_with_annotated.pkl')

In [64]:
df.head()

,Codeursnaam,type_content,doctype_or,political_content,publication_date_x,doc_number,topic_number,words_topic,frames,attrresp,...,doc_num_y,document numer,document nummer,publication_date_y,random,suspicious,text,text_ner,title,YearMonth_y
20,1.0,1,1.0,1.0,2004-09-05,873,2000,"kritiek, betaalde, reis, Kamerleden, omstreden...",1.0,2.0,...,873.0,NaN,873.0,2004-05-09,0.000254,NaN,"Van onze parlementaire redactie DEN HAAG, zon...",NaN,CDA'ers betalen zelf deel Taiwan-reis,200405
21,1.0,1,1.0,1.0,2004-11-22,2222,602,"scholen, basisscholen, wachtlijsten, selectere...",1.0,1.0,...,2222.0,NaN,2222.0,2004-11-22,0.002465,NaN,HIGHLIGHT: Rotterdam houdt strenge selectie a...,NaN,'Opwitten' van zwarte scholen,200411
22,1.0,1,1.0,1.0,2004-11-11,2134,201,"kerken, moskeeën, moslims, Theo van Gogh, Fort...",1.0,2.0,...,2134.0,NaN,2134.0,2004-11-11,0.002500,NaN,"Van onze parlementaire redactie DEN HAAG, don...",NaN,Neerwaartse spiraal baart Kamer zorgen,200411
23,1.0,1,1.0,1.0,2004-11-24,2253,1205,"tbs'ers, tbs-klinieken, instellingen, kliniek",1.0,1.0,...,2253.0,NaN,2253.0,2004-11-24,0.004260,NaN,"Van onze verslaggevers AMSTERDAM, woensdag Tb...",NaN,Pizzakoerier brengt drugs naar tbs'ers,200411
24,1.0,1,1.0,1.0,2005-06-09,1633,1501,"Postbank, afschriften, bank, overschrijfformul...",1.0,1.0,...,1633.0,NaN,NaN,2005-09-06,0.000479,NaN,Van onze parlementaire redactie DEN HAAG - De...,NaN,Kritiek van PvdA op Postbank-plan,200509
